## Imports

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import math
import re

#from bs4 import BeautifulSoup
#from nltk.corpus import stopwords
#from gensim.models import word2vec, Word2Vec

from util.word2vec_as_MF import Word2vecMF
from util.functions import *

import time
dimension = 100
regularization =0.0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read and preprocess enwik9

In [3]:
#real_sentences = load('data/x1')

# Gensim

In [4]:
'''%%time
skip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'''

'%%time\nskip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'

In [5]:
'''skip.get_latest_training_loss()'''

'skip.get_latest_training_loss()'

## Train ro_sgns model starting from SVD of SPPMI

In [6]:
# Create word2vec as matrix factorization model
model_enwik = Word2vecMF()
#model_enwik.data_to_matrices(real_sentences, dimension, 5, 'enwik-200/matrices.npz')
#model_enwik.data_to_matrices(real_sentences, dimension, 5, False)

In [7]:
# If the model has been already created, load it from file
model_enwik.load_matrices(from_file='enwik-200/matrices1.npz')

In [8]:
'''# SVD initialization
SPPMI = np.maximum(np.log(model_enwik.D) - np.log(model_enwik.B), 0)
# print SPPMI
u, s, vt = svds(SPPMI, k=dimension)
C_svd = u.dot(np.sqrt(np.diag(s))).T
W_svd = np.sqrt(np.diag(s)).dot(vt)
print(model_enwik.MF(C_svd, W_svd))'''

'# SVD initialization\nSPPMI = np.maximum(np.log(model_enwik.D) - np.log(model_enwik.B), 0)\n# print SPPMI\nu, s, vt = svds(SPPMI, k=dimension)\nC_svd = u.dot(np.sqrt(np.diag(s))).T\nW_svd = np.sqrt(np.diag(s)).dot(vt)\nprint(model_enwik.MF(C_svd, W_svd))'

In [9]:
'''model_enwik.C = C_svd
model_enwik.W = W_svd
#model_enwik.save_CW('enwik-200/initializations/SVD_dim200', 0)
print(model_enwik.C.shape, model_enwik.W.shape, model_enwik.B.shape, model_enwik.D.shape)'''

"model_enwik.C = C_svd\nmodel_enwik.W = W_svd\n#model_enwik.save_CW('enwik-200/initializations/SVD_dim200', 0)\nprint(model_enwik.C.shape, model_enwik.W.shape, model_enwik.B.shape, model_enwik.D.shape)"

In [10]:
C0, W0, step_size = BFGD_init(model_enwik, dimension=dimension, reg=regularization)
'''X0 = 1/norm((model_enwik.B+model_enwik.D)/4, 'fro')*model_enwik.grad_MF(
    np.zeros([dimension,model_enwik.B.shape[0]]), np.zeros([dimension,model_enwik.B.shape[0]]))
print(X0.shape)
u, s, vt = svds(X0, k=dimension)
C0 = u.dot(np.sqrt(np.diag(s))).T
W0 = np.sqrt(np.diag(s)).dot(vt)
print(model_enwik.MF(C0, W0))'''

(356, 356)
Initial loss (1783763.7762035725, 5.251327505899099e-14) theoretical step size 5.989473446661968e-07


"X0 = 1/norm((model_enwik.B+model_enwik.D)/4, 'fro')*model_enwik.grad_MF(\n    np.zeros([dimension,model_enwik.B.shape[0]]), np.zeros([dimension,model_enwik.B.shape[0]]))\nprint(X0.shape)\nu, s, vt = svds(X0, k=dimension)\nC0 = u.dot(np.sqrt(np.diag(s))).T\nW0 = np.sqrt(np.diag(s)).dot(vt)\nprint(model_enwik.MF(C0, W0))"

In [11]:
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='PS', 
               d=dimension,
               eta = 5e-5,
               MAX_ITER=10,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'PS\', \n               d=dimension,\n               eta = 5e-5,\n               MAX_ITER=10,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [12]:
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='AM', 
               d=dimension,
               eta = 5e-6,
               lbd = 1.0,
               MAX_ITER=189000,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'AM\', \n               d=dimension,\n               eta = 5e-6,\n               lbd = 1.0,\n               MAX_ITER=189000,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [20]:
start_time = time.time()
opt_experiment(model_enwik,
               mode='BFGD',
               d=dimension,
               eta = step_size,
               reg = regularization,
               MAX_ITER=1000,
               from_iter=100,
               start_from='1X0',
               itv_print=100,
               itv_save=100,
               init=(True, C0, W0), display=True)
print("--- %s seconds ---" % (time.time() - start_time))

(1689140.0296356275, 0.0011626787364361353)
DEBUG
Iter #: 200 loss (1590364.3758813497, 0.001681167319522623)
Iter #: 300 loss (1517475.279448201, 0.0013483108683180626)
Iter #: 400 loss (1459610.0542190813, 0.0011155627664101038)
Iter #: 500 loss (1413073.1249633937, 0.0007722982843572582)
Iter #: 600 loss (1375305.6776206899, 0.0006411572326509304)
Iter #: 700 loss (1343839.1515819132, 0.0010583525961904003)
Iter #: 800 loss (1316975.6810543337, 0.0017474710913127985)
Iter #: 900 loss (1293619.4535994006, 0.0025758256525294015)
Iter #: 1000 loss (1273037.1230879151, 0.003486764564724799)
Iter #: 1100 loss (1254719.9129956383, 0.00443615939996299)
--- 9.145961284637451 seconds ---


In [ ]:
'''model = Word2Vec(real_sentences, size=200, window=5, min_count=5, workers=4)
fname = 'original'
model.save(fname)
model = Word2Vec.load(fname)  # you can continue training with the loaded model!'''